### <font color="blue">Import Library  </font>

In [ ]:
import numpy as np
from tensorflow.keras import optimizers
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import cv2
import os
from tensorflow.keras.applications import ResNet50,VGG19
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import KFold

### <font color="blue">Define the Hypeparameters </font>

In [ ]:
NUM_CLASSES = 12
CHANNELS = 3
IMAGE_RESIZE = 224
RESNET50_POOLING_AVERAGE = 'avg'
DENSE_LAYER_ACTIVATION = 'softmax'
OBJECTIVE_FUNCTION = 'categorical_crossentropy'
LOSS_METRICS = ['accuracy']
NUM_EPOCHS = 20
EARLY_STOP_PATIENCE = 3
STEPS_PER_EPOCH_TRAINING = 10
STEPS_PER_EPOCH_VALIDATION = 10
BATCH_SIZE_TRAINING = 100
BATCH_SIZE_VALIDATION = 100
BATCH_SIZE_TESTING = 1

### <font color="blue"> Ininialize the ResNet50 with pretrain weights</font>

In [ ]:
base_model=ResNet50(include_top = False, pooling = RESNET50_POOLING_AVERAGE, weights = 'imagenet')

### <font color="blue"> Define the optimize </font>

In [ ]:
sgd = optimizers.SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)

In [ ]:
image_size = IMAGE_RESIZE


### <font color="blue"> Define the data generator</font>

In [ ]:
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
train_data_dir = "../input/plant-seedlings-classification/train"

In [ ]:
train_generator = data_generator.flow_from_directory(
        train_data_dir,
        target_size=(128, 128),
        batch_size=4750,
        class_mode='categorical')

### <font color="blue"> Define callback for early stop and model checkpoints</font>

In [ ]:
cb_early_stopper = EarlyStopping(monitor = 'loss', patience = EARLY_STOP_PATIENCE)
cb_checkpointer = ModelCheckpoint(filepath = '../working/best.hdf5', monitor = 'loss', save_best_only = True, mode = 'auto')

### <font color="blue"> 5 fold cros validation </font>

In [ ]:
kfold = KFold(n_splits=5, shuffle=True)
X, label = train_generator.next()

### <font  color="blue"> ResNet50(only last 3 layers trainable)+ three dense layers

In [ ]:
scores=[]
for train, test in kfold.split(X, label):
    for layer in base_model.layers[:173]:
        layer.trainable = False
    for layer in base_model.layers[173:]:
        layer.trainable = True
    tf.keras.backend.clear_session()
    model = Sequential()
    model.add(base_model)
    model.add(Dense(1024,"relu"))
    model.add(Dense(512,"relu"))
    model.add(Dense(128,"relu"))
    model.add(Dense(NUM_CLASSES, activation = DENSE_LAYER_ACTIVATION))
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(X[train],label[train],epochs = 20)
    
    score= model.evaluate(X[test], label[test], verbose=0)
    scores.append(score)
    tf.keras.backend.clear_session()
    del model

In [ ]:
accuracy=[i[1] for i in scores]

In [ ]:
accuracy

### <font color="blue">Averaage Cross validation accuracy

In [ ]:
np.mean(accuracy)

### <font color="blue">ResNet50(Last 15 layers trainable)+ three dense layers

In [ ]:
scores=[]
for train, test in kfold.split(X, label):
    for layer in base_model.layers[:160]:
        layer.trainable = False
    for layer in base_model.layers[160:]:
        layer.trainable = True
    tf.keras.backend.clear_session()
    model = Sequential()
    model.add(base_model)
    model.add(Dense(1024,"relu"))
    model.add(Dense(512,"relu"))
    model.add(Dense(128,"relu"))
    model.add(Dense(NUM_CLASSES, activation = DENSE_LAYER_ACTIVATION))
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(X[train],label[train],epochs = 20)
    
    score= model.evaluate(X[test], label[test], verbose=0)
    scores.append(score)
    tf.keras.backend.clear_session()
    del model

In [ ]:
accuracy=[i[1] for i in scores]

### <font color="blue">Average cross validation accuracy

In [ ]:
np.mean(accuracy)

### <font color="blue"> ResNet50(last 15 layers traibable)+Dense layers+ Batch Normalization layers+ Droupout layers

In [ ]:
scores=[]
for train, test in kfold.split(X, label):
    for layer in base_model.layers[:160]:
        layer.trainable = False
    for layer in base_model.layers[160:]:
        layer.trainable = True
    tf.keras.backend.clear_session()
    model = Sequential()
    model.add(base_model)
    model.add(tf.keras.layers.BatchNormalization())
    model.add(Dense(1024,"relu"))
    model.add(tf.keras.layers.Dropout(0.15))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(Dense(512,"relu"))
    model.add(tf.keras.layers.Dropout(0.15))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(Dense(128,"relu"))
    model.add(tf.keras.layers.Dropout(0.15))
    model.add(Dense(NUM_CLASSES, activation = DENSE_LAYER_ACTIVATION))
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(X[train],label[train],epochs = 20,callbacks=[cb_early_stopper])
    
    score= model.evaluate(X[test], label[test], verbose=0)
    scores.append(score)
    tf.keras.backend.clear_session()
    del model

In [ ]:
accuracy=[i[1] for i in scores]

In [ ]:
accuracy

### <font color="blue">Average Cross validation accuracy

In [ ]:
np.mean(accuracy)

### <font color="blue"> Kagggle Submision

In [ ]:
train_generator = data_generator.flow_from_directory(
        train_data_dir,
        target_size=(256, 256),
        batch_size=BATCH_SIZE_TRAINING,
        class_mode='categorical')

In [ ]:
for layer in base_model.layers[:160]:
    layer.trainable = False
for layer in base_model.layers[160:]:
    layer.trainable = True
tf.keras.backend.clear_session()
model = Sequential()
model.add(base_model)
model.add(tf.keras.layers.BatchNormalization())
model.add(Dense(1024,"relu"))
model.add(tf.keras.layers.Dropout(0.15))
model.add(tf.keras.layers.BatchNormalization())
model.add(Dense(512,"relu"))
model.add(tf.keras.layers.Dropout(0.15))
model.add(tf.keras.layers.BatchNormalization())
model.add(Dense(128,"relu"))
model.add(tf.keras.layers.Dropout(0.15))
model.add(Dense(NUM_CLASSES, activation = DENSE_LAYER_ACTIVATION))
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
fit_history = model.fit_generator(
        train_generator,
        steps_per_epoch=STEPS_PER_EPOCH_TRAINING,
        epochs = NUM_EPOCHS,
        callbacks=[cb_checkpointer, cb_early_stopper]
)

In [ ]:
model.load_weights("../working/best.hdf5")

In [ ]:
print(fit_history.history.keys())

In [ ]:
test_generator = data_generator.flow_from_directory(
    directory= "../input/plant-seedlings-classification/",
    target_size = (256, 256),
    batch_size = BATCH_SIZE_TESTING,
    class_mode = 'categorical',
    shuffle = False,
    seed = 123,
    classes=['test']
)

In [ ]:
species_list = ["Black-grass", "Charlock", "Cleavers", "Common Chickweed", "Common wheat", "Fat Hen",
                "Loose Silky-bent", "Maize", "Scentless Mayweed", "Shepherds Purse", "Small-flowered Cranesbill",
                "Sugar beet"]
preds = model.predict(test_generator, steps=test_generator.samples)
class_list = []
for i in range(preds.shape[0]):
    y_class = preds[i,:].argmax(axis=-1)
    class_list.append(species_list[y_class])
    
submission = pd.DataFrame()
submission['file'] = test_generator.filenames
submission['file'] = submission['file'].str.replace(r'test/', '')
submission['species'] = class_list

In [ ]:
preds.shape[0]

In [ ]:
submission.head()

In [ ]:
submission.to_csv('final_submision1.csv', index=False)

In [ ]:
del model

In [ ]:
for layer in base_model.layers[:160]:
    layer.trainable = False
for layer in base_model.layers[160:]:
    layer.trainable = True
tf.keras.backend.clear_session()
model = Sequential()
model.add(base_model)
model.add(Dense(1024,"relu"))
model.add(Dense(512,"relu"))
model.add(Dense(128,"relu"))
model.add(Dense(NUM_CLASSES, activation = DENSE_LAYER_ACTIVATION))
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
fit_history = model.fit_generator(
        train_generator,
        steps_per_epoch=STEPS_PER_EPOCH_TRAINING,
        epochs = NUM_EPOCHS,
        callbacks=[cb_checkpointer, cb_early_stopper]
)

In [ ]:
print(fit_history.history.keys())

In [ ]:
test_generator = data_generator.flow_from_directory(
    directory= "../input/plant-seedlings-classification/",
    target_size = (256, 256),
    batch_size = BATCH_SIZE_TESTING,
    class_mode = 'categorical',
    shuffle = False,
    seed = 123,
    classes=['test']
)

In [ ]:
species_list = ["Black-grass", "Charlock", "Cleavers", "Common Chickweed", "Common wheat", "Fat Hen",
                "Loose Silky-bent", "Maize", "Scentless Mayweed", "Shepherds Purse", "Small-flowered Cranesbill",
                "Sugar beet"]
preds = model.predict(test_generator, steps=test_generator.samples)
class_list = []
for i in range(preds.shape[0]):
    y_class = preds[i,:].argmax(axis=-1)
    class_list.append(species_list[y_class])
    
submission = pd.DataFrame()
submission['file'] = test_generator.filenames
submission['file'] = submission['file'].str.replace(r'test/', '')
submission['species'] = class_list

In [ ]:
preds.shape[0]

In [ ]:
submission.head()

In [ ]:
submission.to_csv('final_submision2.csv', index=False)